Original Essay/Tutorial: https://norvig.com/sudoku.html

Every square in a sodoku puzzle has 3 units: the smaller square, row, and column. With these 3 units, this square has 20 peers. e.g. the square C2:

        A2   |         |                    |         |            A1 A2 A3|         |         
        B2   |         |                    |         |            B1 B2 B3|         |         
        C2   |         |            C1 C2 C3| C4 C5 C6| C7 C8 C9   C1 C2 C3|         |         
    ---------+---------+---------  ---------+---------+---------  ---------+---------+---------
        D2   |         |                    |         |                    |         |         
        E2   |         |                    |         |                    |         |         
        F2   |         |                    |         |                    |         |         
    ---------+---------+---------  ---------+---------+---------  ---------+---------+---------
        G2   |         |                    |         |                    |         |         
        H2   |         |                    |         |                    |         |         
        I2   |         |                    |         |                    |         |         
    
 

In [1]:
def cross(A, B):
    "Cross product of elements in A and elements in B"
    return [a + b for a in A for b in B]

digits = '123456789'
rows = 'ABCDEFGHI'
cols = digits
squares = cross(rows,cols)
print(squares)

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9']


In [2]:
unitlist = ([cross(rows, c) for c in cols]+
           [cross(r,cols) for r in rows]+
           [cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789')])

#  "units is a dictionary where each square maps to the list of units that contain the square"
units = dict((s, [u for u in unitlist if s in u])
            for s in squares)
print(units['A1'])

[['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'], ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'], ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']]


"peers is a dictionary where each square s maps to the set of squares formed by the union of the squares in the units of s, but not s itself"

In [3]:
peers = dict((s, set(sum(units[s], [])) - set([s])) for s in squares)

### Tests

In [4]:
def test():
    "A set of unit tests."
    assert len(squares) == 81
    assert len(unitlist) == 27
    assert all(len(units[s]) == 3 for s in squares)
    assert all(len(peers[s]) == 20 for s in squares)
    assert units['C2'] == [['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
                           ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
                           ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']]
    assert peers['C2'] == set(['A2', 'B2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2',
                               'C1', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
                               'A1', 'A3', 'B1', 'B3'])
    print('All tests pass.')

test()

All tests pass.


"Now that we have squares, units, and peers, the next step is to define the Sudoku playing grid. Actually we need two representations: First, a textual format used to specify the initial state of a puzzle; we will reserve the name grid for this. Second, an internal representation of any state of a puzzle, partially solved or complete; this we will call a values collection because it will give all the remaining possible values for each square. For the textual format (grid) we'll allow a string of characters with 1-9 indicating a digit, and a 0 or period specifying an empty square. All other characters are ignored (including spaces, newlines, dashes, and bars). So each of the following three grid strings represent the same puzzle: "



In [5]:
"4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"

"""
400000805
030000000
000700000
020000060
000080400
000010000
000603070
500200000
104000000"""

"""
4 . . |. . . |8 . 5 
. 3 . |. . . |. . . 
. . . |7 . . |. . . 
------+------+------
. 2 . |. . . |. 6 . 
. . . |. 8 . |4 . . 
. . . |. 1 . |. . . 
------+------+------
. . . |6 . 3 |. 7 . 
5 . . |2 . . |. . . 
1 . 4 |. . . |. . . 
"""

'\n4 . . |. . . |8 . 5 \n. 3 . |. . . |. . . \n. . . |7 . . |. . . \n------+------+------\n. 2 . |. . . |. 6 . \n. . . |. 8 . |4 . . \n. . . |. 1 . |. . . \n------+------+------\n. . . |6 . 3 |. 7 . \n5 . . |2 . . |. . . \n1 . 4 |. . . |. . . \n'

"The value of each key will be the possible digits for that square: a single digit if it was given as part of the puzzle definition or if we have figured out what it must be, and a collection of several digits if we are still uncertain.... A grid where A1 is 7 and C7 is empty would be represented as {'A1': '7', 'C7': '123456789', ...}"

"The function __assign(values, s, d)__ will return the updated _values_ (including the updates from constraint propagation), but if there is a contradiction--if the assignment cannot be made consistently--then __assign__ returns __False__. For example, if a grid starts with the digits '77...' then when we try to assign the 7 to A2, __assign__ would notice that 7 is not a possibility for A2, because it was eliminated by the peer, A1.

It turns out that the fundamental operation is not assigning a value, but rather eliminating one of the possible values for a square, which we implement with __eliminate(values, s, d)__. Once we have __eliminate__, then __assign(values, s, d)__ can be defined as "eliminate all the values from s except __d__". "

In [6]:
def parse_grid(grid):
    ''' Convert grid to a dict of possible values, (square: digits), or
    return False if a contraction is detected
    '''
    # To start, every square can be any digit; then assign values from the grid.
    values = dict((s, digits) for s in squares)
    for s, d in grid_values(grid).items():
        if d in digits and not assign(values, s, d):
            return False ## Fail if we can't assign d to square s
    return values

def grid_values(grid):
    "Convert grid into a dict  of (square: char) with '0' or '.' for empties."
    chars = [c for c in grid if c in digits or c in '0.']
    assert len(chars) == 81
    return dict(zip(squares, chars))

def assign(values, s, d):
    '''
    Eliminate all the other values (except d) from values[s] and propagate.
    Returns values, except return False if a contracdiction is detected.
    '''
    other_values = values[s].replace(d, '')
    if all(eliminate(values, s, d2) for d2 in other_values):
        return values
    else:
        return False
    
def eliminate(values, s, d):
    '''
    Eliminate d from from values[s]; propagate when values or places <= 2.
    Return values, except return False if a contradiction is detected.
    '''
    
    if d not in values[s]:
        return values ## d is already eliminated
    # otherwise eliminate d
    values[s] = values[s].replace(d, '')
    ## (1) If a square s is reduced to one value d2, then eliminate d2 from the peers.
    if len(values[s]) == 0:
        return False ## Contradiction: remove last value
    elif len(values[s]) == 1:
        d2 = values[s]
        # return false if d2 is not available as a value for peers
        if not all(eliminate(values, s2, d2) for s2 in peers[s]):
            return False
    ## (2) If a unit is reduced to only one place for a value d, then put it there.
    for u in units[s]:
        dplace = [s for s in u if d in values[s]]
        if len(dplace) == 0:
            return False ## Contraction: no place for this value
        elif len(dplace) == 1:
            # d can only be in one place in unit; assign it there
            if not assign(values, dplace[0], d):
                return False
    return values

In [7]:
grid1 = '003020600900305001001806400008102900700000008006708200002609500800203009005010300'
print(grid_values(grid1))

{'A1': '0', 'A2': '0', 'A3': '3', 'A4': '0', 'A5': '2', 'A6': '0', 'A7': '6', 'A8': '0', 'A9': '0', 'B1': '9', 'B2': '0', 'B3': '0', 'B4': '3', 'B5': '0', 'B6': '5', 'B7': '0', 'B8': '0', 'B9': '1', 'C1': '0', 'C2': '0', 'C3': '1', 'C4': '8', 'C5': '0', 'C6': '6', 'C7': '4', 'C8': '0', 'C9': '0', 'D1': '0', 'D2': '0', 'D3': '8', 'D4': '1', 'D5': '0', 'D6': '2', 'D7': '9', 'D8': '0', 'D9': '0', 'E1': '7', 'E2': '0', 'E3': '0', 'E4': '0', 'E5': '0', 'E6': '0', 'E7': '0', 'E8': '0', 'E9': '8', 'F1': '0', 'F2': '0', 'F3': '6', 'F4': '7', 'F5': '0', 'F6': '8', 'F7': '2', 'F8': '0', 'F9': '0', 'G1': '0', 'G2': '0', 'G3': '2', 'G4': '6', 'G5': '0', 'G6': '9', 'G7': '5', 'G8': '0', 'G9': '0', 'H1': '8', 'H2': '0', 'H3': '0', 'H4': '2', 'H5': '0', 'H6': '3', 'H7': '0', 'H8': '0', 'H9': '9', 'I1': '0', 'I2': '0', 'I3': '5', 'I4': '0', 'I5': '1', 'I6': '0', 'I7': '3', 'I8': '0', 'I9': '0'}


Displaying the puzzle:

In [8]:
def display(values):
    "Display these values as a 2-D grid."
    width = 1+max(len(values[s]) for s in squares)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)


In [9]:
grid1 = '003020600900305001001806400008102900700000008006708200002609500800203009005010300'

display(parse_grid(grid1))

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 


Sometimes rote application of the above Constraint application is not enough with a really hard problem:

In [10]:
grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
display(parse_grid(grid2))

   4      1679   12679  |  139     2369    269   |   8      1239     5    
 26789     3    1256789 | 14589   24569   245689 | 12679    1249   124679 
  2689   15689   125689 |   7     234569  245689 | 12369   12349   123469 
------------------------+------------------------+------------------------
  3789     2     15789  |  3459   34579    4579  | 13579     6     13789  
  3679   15679   15679  |  359      8     25679  |   4     12359   12379  
 36789     4     56789  |  359      1     25679  | 23579   23589   23789  
------------------------+------------------------+------------------------
  289      89     289   |   6      459      3    |  1259     7     12489  
   5      6789     3    |   2      479      1    |   69     489     4689  
   1      6789     4    |  589     579     5789  | 23569   23589   23689  


## Search 
"What is the search algorithm? Simple: first make sure we haven't already found a solution or a contradiction, and if not, choose one unfilled square and consider all its possible values. One at a time, try assigning the square each value, and searching from the resulting position. In other words, we search for a value d such that we can successfully search for a solution from the result of assigning square s to d. If the search leads to an failed position, go back and consider another value of d. This is a recursive search, and we call it a depth-first search because we (recursively) consider all possibilities under __values[s] = d__ before we consider a different value for s. "

Using depth first search and a heuristic called minimum remaining value:

In [11]:
def solve(grid): return search(parse_grid(grid))

def search(values):
    "Using depth-first search and propagation, try all possible values."
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in squares): 
        return values ## Solved!
    ## Chose the unfilled square s with the fewest possibilities
    n,s = min((len(values[s]), s) for s in squares if len(values[s]) > 1)
    return some(search(assign(values.copy(), s, d)) for d in values[s])

def some(seq):
    "Return some element of seq that is true."
    for e in seq:
        if e: return e
    return False

In [12]:
hard1  = '.....6....59.....82....8....45........3........6..3.54...325..6..................'
solve(hard1)

{'A1': '4',
 'A2': '3',
 'A3': '8',
 'A4': '7',
 'A5': '9',
 'A6': '6',
 'A7': '2',
 'A8': '1',
 'A9': '5',
 'B1': '6',
 'B2': '5',
 'B3': '9',
 'B4': '1',
 'B5': '3',
 'B6': '2',
 'B7': '4',
 'B8': '7',
 'B9': '8',
 'C1': '2',
 'C2': '7',
 'C3': '1',
 'C4': '4',
 'C5': '5',
 'C6': '8',
 'C7': '6',
 'C8': '9',
 'C9': '3',
 'D1': '8',
 'D2': '4',
 'D3': '5',
 'D4': '2',
 'D5': '1',
 'D6': '9',
 'D7': '3',
 'D8': '6',
 'D9': '7',
 'E1': '7',
 'E2': '1',
 'E3': '3',
 'E4': '5',
 'E5': '6',
 'E6': '4',
 'E7': '8',
 'E8': '2',
 'E9': '9',
 'F1': '9',
 'F2': '2',
 'F3': '6',
 'F4': '8',
 'F5': '7',
 'F6': '3',
 'F7': '1',
 'F8': '5',
 'F9': '4',
 'G1': '1',
 'G2': '9',
 'G3': '4',
 'G4': '3',
 'G5': '2',
 'G6': '5',
 'G7': '7',
 'G8': '8',
 'G9': '6',
 'H1': '3',
 'H2': '6',
 'H3': '2',
 'H4': '9',
 'H5': '8',
 'H6': '7',
 'H7': '5',
 'H8': '4',
 'H9': '1',
 'I1': '5',
 'I2': '8',
 'I3': '7',
 'I4': '6',
 'I5': '4',
 'I6': '1',
 'I7': '9',
 'I8': '3',
 'I9': '2'}